# Predicting Malicious Cyber Connections
<p style="margin:30px">
    <img style="display:inline; margin-right:50px" width=50% src="https://www.featuretools.com/wp-content/uploads/2017/12/FeatureLabs-Logo-Tangerine-800.png" alt="Featuretools" />
</p>

The general setup for the problem is a common one: we have a single table of log lines recording Internet traffic between various sources. Traffic between a source and destination is labeled as malicious or clean in the dataset, and we'd like to be able to predict ahead of time if a future connection between a source and a destination will be malicious.

We'll demonstrate an end-to-end workflow using this [Cybersecurity Dataset](). This notebook demonstrates a rapid way to predict whether a connection (defined as a source name/destination name pair) is malicious.


## Highlights
* Quickly make end-to-end workflow using log-line cybersecurity data
* Find interesting automatically generated features

Note: this is an extremely imbalanced dataset, and would benefit tremendously from additional positive (malicious) labels

In [20]:
import featuretools as ft
from featuretools.primitives import CumMean, Percentile
from featuretools.selection import remove_low_information_features
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
import utils

# Step 1: Understanding the Data
Here we load in the data and do a bit of preprocessing

In [21]:
cyber_df = pd.read_csv("CyberFLTenDays.csv")
cyber_df.index.name = "log_id"
cyber_df.reset_index(inplace=True, drop=False)
cyber_df['label'] = cyber_df['label'].map({'N': False, 'A': True}, na_action='ignore')

# Sample down negative examples because very few positives
# Can also do this after the feature engineering step (but doing it here reduces computation time)
cyber_df_pos = cyber_df[cyber_df['label']]
cyber_df_neg = cyber_df[~cyber_df['label']].sample(100000)
cyber_df = pd.concat([cyber_df_pos, cyber_df_neg]).sort_values(['secs'])

In [22]:
cyber_df.head()

,log_id,secs,src_name,dest_name,src_host,dest_host,auth_type,login_type,stage,result,label
4,4,4,C1034$@DOM1,SYSTEM@C1034,C1034,C1034,Negotiate,Service,LogOn,Success,False
8,8,18,U119@DOM1,U119@DOM1,C2143,C1790,?,?,TGS,Success,False
10,10,25,U175@DOM1,U175@DOM1,C1085,C1085,?,?,TGS,Success,False
15,15,70,C860$@DOM1,C860$@DOM1,C1798,C860,Kerberos,Network,LogOn,Success,False
16,16,74,C1697$@DOM1,C1697$@DOM1,C529,C529,?,Network,LogOff,Success,False


## Create an EntitySet
To apply Deep Feature Synthesis we need to establish an `EntitySet` structure for our data. Since we're interested in predicting for combinations of "src_name" and "dest_name" (we call this pair a "session"), we need to create a separate normalized entity for "sessions".

In [24]:
es = ft.EntitySet("CyberLL")
# create an index column
cyber_df["name_host_pair"] = cyber_df["src_name"].str.cat(
                                [cyber_df["dest_name"],
                                 cyber_df["src_host"],
                                 cyber_df["dest_host"]],
                                sep=' / ')
cyber_df["session_id"] = cyber_df["src_name"].str.cat(
                                 cyber_df["dest_name"],
                                 sep=' / ')

es.entity_from_dataframe("log",
                         cyber_df,
                         index="log_id",
                         time_index="secs")
es.normalize_entity(base_entity_id="log",
                    new_entity_id="name_host_pairs",
                    index="name_host_pair",
                    additional_variables=["src_name", "dest_name",
                                          "src_host", "dest_host",
                                          #"src_pair",
                                          #"dest_pair",
                                          "session_id",
                                          "label"])
es.normalize_entity(base_entity_id="name_host_pairs",
                    new_entity_id="sessions",
                    index="session_id",
                    additional_variables=["dest_name", "src_name"])

Entityset: CyberLL
  Entities:
    log (shape = [100329, 7])
    name_host_pairs (shape = [61964, 6])
    sessions (shape = [19234, 4])
  Relationships:
    log.name_host_pair -> name_host_pairs.name_host_pair
    name_host_pairs.session_id -> sessions.session_id

In [25]:
cyber_df.head()

,log_id,secs,src_name,dest_name,src_host,dest_host,auth_type,login_type,stage,result,label,name_host_pair,session_id
4,4,4,C1034$@DOM1,SYSTEM@C1034,C1034,C1034,Negotiate,Service,LogOn,Success,False,C1034$@DOM1 / SYSTEM@C1034 / C1034 / C1034,C1034$@DOM1 / SYSTEM@C1034
8,8,18,U119@DOM1,U119@DOM1,C2143,C1790,?,?,TGS,Success,False,U119@DOM1 / U119@DOM1 / C2143 / C1790,U119@DOM1 / U119@DOM1
10,10,25,U175@DOM1,U175@DOM1,C1085,C1085,?,?,TGS,Success,False,U175@DOM1 / U175@DOM1 / C1085 / C1085,U175@DOM1 / U175@DOM1
15,15,70,C860$@DOM1,C860$@DOM1,C1798,C860,Kerberos,Network,LogOn,Success,False,C860$@DOM1 / C860$@DOM1 / C1798 / C860,C860$@DOM1 / C860$@DOM1
16,16,74,C1697$@DOM1,C1697$@DOM1,C529,C529,?,Network,LogOff,Success,False,C1697$@DOM1 / C1697$@DOM1 / C529 / C529,C1697$@DOM1 / C1697$@DOM1


# Generate labels and associated cutoff times

Featuretools can generate features for each session strictly before an associated cutoff time. We find these cutoff times in the process of computing labels. Labels are defined as follows:

For a given session:
 * After seeing the same name/host pair N times
 * Predict L observations of this same session in the future
 * Where any connections from this session in a window of size W are malicious
 
We will set N = 2 (number of observations to wait for), L = 2 (lead time), and W = 10 (prediction window)

In [4]:
def generate_cutoffs(cyber_df, index_col, after_n_obs, lead, prediction_window):
    window_start = after_n_obs + lead
    window_end = window_start + prediction_window
    grouped = cyber_df.groupby(index_col)[index_col].count()
    grouped.name = "count"
    min_obs = after_n_obs + lead + 1
    enough_examples = grouped[grouped > min_obs].to_frame().reset_index()
    enough_examples = cyber_df[cyber_df[index_col].isin(enough_examples[index_col])]
    def get_label_and_cutoff(df):
        cutoff = df.iloc[after_n_obs]
        cutoff['label'] = df.iloc[window_start: window_end]["label"].any()
        return cutoff
    cutoffs = enough_examples.groupby(index_col)[[index_col, "secs", "label"]].apply(get_label_and_cutoff)
    return cutoffs

In [5]:
cutoffs = generate_cutoffs(cyber_df, "session_id", 2, 2, 10)

In [6]:
cutoffs['label'].value_counts()

False    4113
True       35
Name: label, dtype: int64

# Compute features using DFS

In [7]:
fm, fl = ft.dfs(entityset=es, target_entity="name_host_pairs", cutoff_time=cutoffs,
                cutoff_time_in_index=True,
                verbose=True, max_depth=3)

Built 54 features
Elapsed: 11:28 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 4138/4138 cutoff times


In [8]:
fm.head()

,,dest_name,src_name,COUNT(name_host_pairs),PERCENT_TRUE(name_host_pairs.label),NUM_UNIQUE(name_host_pairs.src_host),NUM_UNIQUE(name_host_pairs.dest_host),MODE(name_host_pairs.src_host),MODE(name_host_pairs.dest_host),COUNT(log),NUM_UNIQUE(log.auth_type),...,MEAN(name_host_pairs.NUM_UNIQUE(log.result)),NUM_UNIQUE(name_host_pairs.MODE(log.auth_type)),NUM_UNIQUE(name_host_pairs.MODE(log.login_type)),NUM_UNIQUE(name_host_pairs.MODE(log.stage)),NUM_UNIQUE(name_host_pairs.MODE(log.result)),MODE(name_host_pairs.MODE(log.auth_type)),MODE(name_host_pairs.MODE(log.login_type)),MODE(name_host_pairs.MODE(log.stage)),MODE(name_host_pairs.MODE(log.result)),label
session_id,time,,,,,,,,,,,,,,,,,,,,,
C567$@DOM1 / C567$@DOM1,1970-01-01 00:00:00.000000669,C567$@DOM1,C567$@DOM1,2,0.0,2,2,C523,C523,3,2,...,1.0,2,1,2,1,?,Network,LogOff,Success,NaN
C599$@DOM1 / C599$@DOM1,1970-01-01 00:00:00.000000860,C599$@DOM1,C599$@DOM1,3,0.0,1,3,C1619,C101,3,2,...,1.0,2,2,2,1,Kerberos,Network,LogOn,Success,NaN
U66@DOM1 / U66@DOM1,1970-01-01 00:00:00.000002234,U66@DOM1,U66@DOM1,3,0.0,3,3,C1971,C1971,3,2,...,1.0,2,1,2,1,?,Network,LogOff,Success,NaN
C743$@DOM1 / C743$@DOM1,1970-01-01 00:00:00.000002362,C743$@DOM1,C743$@DOM1,2,0.0,2,1,C586,C586,3,2,...,1.0,2,1,2,1,?,Network,LogOff,Success,NaN
U22@DOM1 / U22@DOM1,1970-01-01 00:00:00.000002526,U22@DOM1,U22@DOM1,3,0.0,3,3,C2106,C2106,3,2,...,1.0,2,1,2,1,Kerberos,Network,LogOn,Success,NaN


### Sort indexes to line up cutoffs with feature matrix

In [9]:
fm = fm.reorder_levels(['time', 'session_id']).sort_index()
cutoffs = cutoffs.set_index('secs', append=True).reorder_levels(['secs', 'session_id']).sort_index()
fm['label'] = cutoffs['label'].values

### One-Hot-Encode categorical features and remove features with low information

In [10]:
fm_encoded, fl_encoded = ft.encode_features(fm, fl)
fm_encoded, fl_encoded = remove_low_information_features(fm_encoded, fl_encoded)

# Machine Learning

Now that we have a feature matrix and associated labels, we can build a standard machine learning pipeline with a RandomForestClassifier

First, split up the data into train and test sets

In [15]:
train, test = train_test_split(fm_encoded, test_size=0.2, shuffle=True)

In [16]:
X_train = train
y_train = X_train.pop('label')
X_test = test
y_test = X_test.pop('label')

### Construct the model

In [17]:
imputer = Imputer(missing_values='NaN', strategy="mean", axis=0)
scaler = StandardScaler()
clf = RandomForestClassifier(n_jobs=-1)
model = Pipeline([("imputer", imputer),
                  ("scaler", scaler),
                  ("rf", clf)])

### Fit the model, then score it

In [18]:
model.fit(X_train, y_train)
    
preds = model.predict(X_test)
score = roc_auc_score(preds, y_test)
print('ROC AUC Score: {:.2f}'.format(score))

ROC AUC Score: 0.66


## View the most important features
according to the Random Forest

In [19]:
high_imp_feats = utils.feature_importances(X_train, clf, feats=10)

1: PERCENT_TRUE(name_host_pairs.label) [0.325]
2: NUM_UNIQUE(name_host_pairs.dest_host) [0.077]
3: NUM_UNIQUE(name_host_pairs.src_host) [0.072]
4: MODE(name_host_pairs.MODE(log.auth_type)) = NTLM [0.068]
5: MODE(name_host_pairs.dest_host) = unknown [0.067]
6: MODE(name_host_pairs.src_host) = unknown [0.043]
7: MODE(name_host_pairs.dest_host) = C1065 [0.036]
8: NUM_UNIQUE(log.login_type) [0.036]
9: MODE(name_host_pairs.dest_host) = C529 [0.021]
10: MODE(name_host_pairs.dest_host) = C528 [0.020]
-----

